In [2]:
import pandas as pd
from keras.models import Sequential
from keras.layers import *
from keras.utils import np_utils
import keras
from keras import losses, optimizers, metrics

In [3]:
class ProteinLigandDataGenerator(keras.utils.Sequence):
    def __init__(self, csv_path, batch_size, shuffle=True, dim=(24,24,24), n_channels=2):
        self.df = self.prepare_df(csv_path)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = None
        self.dim = dim
        self.n_channels = n_channels
        self.on_epoch_end()
    
    def prepare_df(self, csv_path):
        # oversampling happens here, 
        match_count = 1
        random_count = 4
        supplement = random_count - match_count
        
        df = pd.read_csv(csv_path)
        same_idx = df[df['pro_id'] == df['lig_id']]
        duplicates = [same_idx]*supplement
        return df.append(duplicates, ignore_index=True)
        
    
    def __len__(self):
        '''
        Batches per epoch
        '''
        return int(len(self.df.index)/self.batch_size)
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size : (index+1)*self.batch_size]
        X, y = self.generate_data(indexes)
        
        return X, y
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df.index))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def generate_data(self, indexes):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        
        sub_df = self.df.iloc[indexes]
        sub_df.reset_index(inplace=True)
        for row in sub_df.itertuples():
            idx = row[0]
            full_X = np.load(row.dests)
            X[idx,:,:,:,:] = full_X
            y[idx] = row.score
            
        return X, y

In [4]:
train_generator = ProteinLigandDataGenerator('./data/csv/train_lig_pro_pairs.csv', 12)
test_generator = ProteinLigandDataGenerator('./data/csv/test_lig_pro_pairs.csv', 12)

In [5]:
model = Sequential()
model.add(Conv3D(96, kernel_size=3, activation='relu', input_shape=(24,24,24,2)))
model.add(BatchNormalization())
model.add(Conv3D(128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool3D())
model.add(Conv3D(196, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv3D(256, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling3D())
model.add(Dense(1, activation='sigmoid'))

sgd = optimizers.SGD(lr=1, decay=1e-5)
model.compile(loss=losses.mean_squared_error, optimizer=sgd, metrics=[metrics.mse])

In [7]:
real_epochs = 3
iterations = real_epochs * len(train_generator)

validation_frequency = 400
epochs = int(iterations/validation_frequency)

for i in range(epochs):
    print(i)
    history = model.fit_generator(train_generator, epochs=1, 
                                  steps_per_epoch=validation_frequency,
                                  use_multiprocessing=True, workers=4, 
                                  verbose=1, validation_data=test_generator)
    model.save(f'models/try_epochs_{i}.h5')

Epoch 1/1
400/400 [==============================] - 72s 179ms/step - loss: 0.0739 - mean_squared_error: 0.0739 - val_loss: 0.0663 - val_mean_squared_error: 0.0663
Epoch 1/1
400/400 [==============================] - 70s 175ms/step - loss: 0.0733 - mean_squared_error: 0.0733 - val_loss: 0.0633 - val_mean_squared_error: 0.0633
Epoch 1/1
Epoch 1/1
400/400 [==============================] - 69s 173ms/step - loss: 0.0489 - mean_squared_error: 0.0489 - val_loss: 0.3774 - val_mean_squared_error: 0.3774
Epoch 1/1
400/400 [==============================] - 69s 173ms/step - loss: 0.0366 - mean_squared_error: 0.0366 - val_loss: 0.0481 - val_mean_squared_error: 0.0481
Epoch 1/1
400/400 [==============================] - 70s 175ms/step - loss: 0.0335 - mean_squared_error: 0.0335 - val_loss: 0.0353 - val_mean_squared_error: 0.0353
Epoch 1/1
400/400 [==============================] - 70s 174ms/step - loss: 0.0280 - mean_squared_error: 0.0280 - val_loss: 0.0278 - val_mean_squared_error: 0.0278
Epoch 

In [8]:
best_fake_epochs = 12
best_num_epochs = validation_frequency * best_fake_epochs / len(train_generator)

In [9]:
best_num_epochs

2.665186007773459

In [10]:
for i in range(epochs, epochs+8):
    print(i)
    history = model.fit_generator(train_generator, epochs=1, 
                                  steps_per_epoch=validation_frequency,
                                  use_multiprocessing=True, workers=4, 
                                  verbose=1, validation_data=test_generator)
    model.save(f'models/try_epochs_{i}.h5')

13
Epoch 1/1
400/400 [==============================] - 70s 174ms/step - loss: 0.0238 - mean_squared_error: 0.0238 - val_loss: 0.2685 - val_mean_squared_error: 0.2685
14
Epoch 1/1
400/400 [==============================] - 71s 176ms/step - loss: 0.0186 - mean_squared_error: 0.0186 - val_loss: 0.0253 - val_mean_squared_error: 0.0253
15
Epoch 1/1
400/400 [==============================] - 70s 176ms/step - loss: 0.0232 - mean_squared_error: 0.0232 - val_loss: 0.1716 - val_mean_squared_error: 0.1716
16
Epoch 1/1
400/400 [==============================] - 71s 177ms/step - loss: 0.0199 - mean_squared_error: 0.0199 - val_loss: 0.0162 - val_mean_squared_error: 0.0162
17
Epoch 1/1
400/400 [==============================] - 69s 173ms/step - loss: 0.0179 - mean_squared_error: 0.0179 - val_loss: 0.0448 - val_mean_squared_error: 0.0448
18
Epoch 1/1
400/400 [==============================] - 70s 176ms/step - loss: 0.0181 - mean_squared_error: 0.0181 - val_loss: 0.0205 - val_mean_squared_error: 0.020